In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import plotly.express as px

In [2]:
# Load .env
load_dotenv()

# Load API-keys
ALPHA_API_KEY = os.getenv('ALPHAVANTAGE_API_KEY')
GBQ_API_KEY = os.getenv('GBQ_API_KEY')

In [3]:
# Initialize DataFrames
my_stocks = pd.DataFrame(columns=['Date','Symbol','Price','Price_Perc_Day_Change','Perc_Change_From_Initial_Price'])
my_stocks_max_price = pd.DataFrame(columns=['Date','Symbol','Price','Price_Perc_Day_Change','Perc_Change_From_Initial_Price'])
my_stocks_initial_value = pd.DataFrame(columns=['Symbol','Initial Date', 'Initial Price'])

In [4]:
# Set the symbols I want to request
symbols = ['PRNHX', 'MFEJX', 'VFIAX','JUSRX']

In [5]:
for symbol in symbols:

    # Get closing trading data per symbol with my API-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={ALPHA_API_KEY}'
    response = requests.get(url)
    symbol_data = response.json()

    trading_data = symbol_data['Time Series (Daily)']

    close_data_series = pd.to_numeric(pd.DataFrame(trading_data).T['4. close'].rename(symbol))


    # Get minimum date per stock and append to my_stocks_initial_value
    initial_date = close_data_series.index.min()
    initial_price = close_data_series[initial_date]

    my_stocks_initial_value = my_stocks_initial_value.append({'Symbol':symbol, 'Initial Date':initial_date, 'Initial Price':initial_price}, ignore_index=True)


    # Create and add series of evolution of price percentage increase as column to my_stocks
    this_symbol_perc_chg = round(((close_data_series / initial_price) - 1) * 100, 2)


    # Bring together data
    this_stock_df = pd.DataFrame(close_data_series).reset_index().rename(columns={'index':'Date',f'{symbol}':'Price'})
    this_stock_df['Symbol'] = symbol
    this_stock_df['Price_Perc_Day_Change'] = round(((this_stock_df['Price'] / this_stock_df['Price'].shift(-1)) - 1) * 100, 2)
    this_stock_df['Price_Perc_Day_Change'].fillna(0, inplace=True)
    this_stock_df['Perc_Change_From_Initial_Price'] = this_symbol_perc_chg.array

    # Append data to my_stocks
    my_stocks = my_stocks.append(this_stock_df)

In [6]:
# Calculate max price per stock and append to my_stocks_max_price
for symbol in symbols:
    my_stocks_max_price = my_stocks_max_price.append(my_stocks.iloc[my_stocks.query(f'Symbol == "{symbol}"')['Price'].idxmax(),:])

In [7]:
my_stocks

,Date,Symbol,Price,Price_Perc_Day_Change,Perc_Change_From_Initial_Price
0,2022-05-25,PRNHX,47.37,1.67,91.70
1,2022-05-24,PRNHX,46.59,-3.14,88.55
2,2022-05-23,PRNHX,48.10,0.67,94.66
3,2022-05-20,PRNHX,47.78,0.17,93.36
4,2022-05-19,PRNHX,47.70,2.89,93.04
...,...,...,...,...,...
4032,2006-05-19,JUSRX,11.24,0.36,-2.09
4033,2006-05-18,JUSRX,11.20,-0.71,-2.44
4034,2006-05-17,JUSRX,11.28,-1.66,-1.74
4035,2006-05-16,JUSRX,11.47,-0.09,-0.09


In [8]:
my_stocks_initial_value

,Symbol,Initial Date,Initial Price
0,PRNHX,1999-11-01,24.71
1,MFEJX,2005-03-31,30.11
2,VFIAX,2000-11-13,124.88
3,JUSRX,2006-05-15,11.48


In [9]:
my_stocks_max_price

,Date,Symbol,Price,Price_Perc_Day_Change,Perc_Change_From_Initial_Price
136,2021-11-09,PRNHX,100.93,0.52,308.46
128,2021-11-19,PRNHX,97.43,-0.45,294.29
99,2022-01-03,PRNHX,75.90,-1.18,207.16
114,2021-12-10,PRNHX,89.27,-0.70,261.27


In [10]:
# Save results to .csv's
my_stocks.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks')
my_stocks_initial_value.to_csv(r'C:\Users\Noah\Desktop\API-Cloud-Stocks\my_stocks_initial_Value')